In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ]
])

y_init = np.array([0.53899612, 0.42058624, -0.06562362, 0.29399291, 
                   0.21496451, 0.02310555, 0.24461934, 0.03874902, 
                   -0.01385762, 0.61120522])

In [ ]:
# Add the new observation
X_new = np.array([[0.365139, 0.474667]])
y_new = np.array([-0.006080823079048435])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

In [ ]:
print(X_init)
print(X_init.shape)   # Useful if it’s an array
print(type(X_init)) 

In [ ]:
print(y_init)
print(y_init.shape)   # Useful if it’s an array
print(type(y_init)) 

# Week 1 

In [ ]:


# --- Fit a Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = sigma[0]
    mu = mu[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    from scipy.stats import norm
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we will minimize

# --- Optimize acquisition function ---
y_best = y_init.max()
bounds = [(0,1), (0,1)]

res = minimize(lambda x: expected_improvement(x, gp, y_best),
               x0=np.random.rand(2),
               bounds=bounds,
               method='L-BFGS-B')

x_next = res.x
print("Next point to evaluate:", x_next)


# Week 2

In [ ]:
# slightly increase alpha (the noise level) since the function outputs are noisy
# --- Fit a Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True)
gp.fit(X_init, y_init)

In [ ]:
# --- Expected Improvement acquisition function ---
# change xi to 0.01
def expected_improvement(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = sigma[0]
    mu = mu[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    from scipy.stats import norm
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we will minimize

In [ ]:
from scipy.optimize import minimize
import numpy as np

y_best = y_all.max()
bounds = [(0, 1), (0, 1)]

def propose_location(acquisition, gp, y_best, bounds, n_restarts=25):
    best_x = None
    best_acq = 1e10
    for i in range(n_restarts):
        x0 = np.random.rand(2)
        res = minimize(lambda x: acquisition(x, gp, y_best),
                       x0=x0,
                       bounds=bounds,
                       method='L-BFGS-B')
        if res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    return best_x

x_next = propose_location(expected_improvement, gp, y_best, bounds)
print("Next point to evaluate:", x_next)
